#Generation d'une simulation de combat

In [33]:
import numpy as np
import random

!pip install matplotlib
import matplotlib.pyplot as plt

!pip install faker
from faker import Faker

## Création des humains

In [34]:
import numpy as np
import random
from faker import Faker

class Human:
    _counter = 0
    STATS_REGION = {
        'Europe_Amerique_Nord': {'locales': ['fr_FR', 'en_US', 'de_DE', 'en_GB'], 'taille_h': 178, 'taille_f': 164, 'imc_moyen': 26.5, 'eco_factor': 45000, 'prenoms_h': ['Jean', 'Pierre', 'Luc', 'John', 'Michael'], 'prenoms_f': ['Marie', 'Sophie', 'Claire', 'Emma', 'Sarah']},
        'Asie_Est': {'locales': ['ja_JP', 'ko_KR', 'zh_CN'], 'taille_h': 172, 'taille_f': 159, 'imc_moyen': 23.0, 'eco_factor': 25000, 'prenoms_h': ['Takeshi', 'Kenji', 'Wei'], 'prenoms_f': ['Sakura', 'Yui', 'Mei']},
        'Asie_Sud': {'locales': ['hi_IN', 'en_IN'], 'taille_h': 165, 'taille_f': 152, 'imc_moyen': 22.0, 'eco_factor': 6000, 'prenoms_h': ['Raj', 'Amit', 'Arjun'], 'prenoms_f': ['Priya', 'Ananya', 'Deepika']},
        'Afrique': {'locales': ['fr_MA', 'en_NG', 'sw_KE'], 'taille_h': 170, 'taille_f': 158, 'imc_moyen': 23.5, 'eco_factor': 5000, 'prenoms_h': ['Mohammed', 'Amadou', 'Kofi'], 'prenoms_f': ['Fatou', 'Amina', 'Ngozi']},
        'Amerique_Latine': {'locales': ['es_MX', 'es_AR', 'pt_BR'], 'taille_h': 172, 'taille_f': 160, 'imc_moyen': 27.0, 'eco_factor': 15000, 'prenoms_h': ['Carlos', 'Jose', 'Luis'], 'prenoms_f': ['Maria', 'Sofia', 'Isabella']},
        'Oceanie': {'locales': ['en_AU', 'en_NZ'], 'taille_h': 176, 'taille_f': 163, 'imc_moyen': 27.5, 'eco_factor': 42000, 'prenoms_h': ['Jack', 'Liam', 'Oliver'], 'prenoms_f': ['Charlotte', 'Olivia', 'Mia']}
    }
    CLASSES_SOCIALES = ['Indigent', 'Misérable', 'Pauvre', 'Modeste', 'Classe moyenne inf.', 'Classe moyenne sup.', 'Aisé', 'Riche', 'Très riche', 'Fortune']

    def __init__(self):
        Human._counter += 1
        self.id = Human._counter
        poids_regions = [0.15, 0.30, 0.25, 0.15, 0.10, 0.05]
        self.region_key = random.choices(list(self.STATS_REGION.keys()), weights=poids_regions, k=1)[0]
        region_data = self.STATS_REGION[self.region_key]

        self.sexe = np.random.choice(['Homme', 'Femme'], p=[0.505, 0.495])
        self._generer_identite(region_data)
        self._calculer_age()
        self._calculer_physique(region_data)
        self._calculer_sante()

        # --- AJOUT ICI ---
        self._calculer_mental()

        self._calculer_stats_combat()
        self._calculer_entrainement()
        self._calculer_richesse(region_data)
        self._initialiser_corps()
        self._verifier_capacite_combat()

    def _generer_identite(self, region_data):
        if self.sexe == 'Homme': self.prenom = random.choice(region_data['prenoms_h'])
        else: self.prenom = random.choice(region_data['prenoms_f'])

    def _calculer_age(self):
        choix = random.choices(['enfant', 'adulte', 'mature', 'vieux'], weights=[0.30, 0.35, 0.25, 0.10], k=1)[0]
        if choix == 'enfant': self.age = random.randint(0, 17)
        elif choix == 'adulte': self.age = random.randint(18, 40)
        elif choix == 'mature': self.age = random.randint(41, 65)
        else: self.age = random.randint(66, 100)

    def _calculer_physique(self, region_data):
        base_taille = region_data['taille_h'] if self.sexe == 'Homme' else region_data['taille_f']
        facteur_croissance = min(1.0, self.age / 18)
        if self.age < 18: facteur_croissance = 0.5 + (self.age / 18 * 0.5)
        self.taille = base_taille * facteur_croissance + np.random.normal(0, 3)
        imc_target = region_data['imc_moyen']
        if self.age < 18: imc_target = 16 + (self.age * 0.4)
        elif self.age > 60: imc_target *= 1.05
        self.poids = round(np.random.normal(imc_target, 2) * (self.taille/100)**2, 1)

    def _calculer_sante(self):
        proba_maladie = 0.05 + (self.age / 100) * 0.75
        self.est_malade = random.random() < proba_maladie
        self.severite_maladie = min(1.0, np.random.exponential(0.3)) if self.est_malade else 0.0

    # --- NOUVELLE METHODE ---
    def _calculer_mental(self):
        base = np.random.normal(0.85, 0.15) # La plupart sont stables
        if self.age < 12: base -= 0.2
        if self.age > 80: base -= 0.1
        if self.est_malade: base -= self.severite_maladie * 0.3
        self.mental = max(0.1, min(1.0, base))
        self.mental_max = self.mental

    def _calculer_stats_combat(self):
        self.force = np.random.normal(30, 10) + (self.poids * 0.4)
        facteur_age_vitesse = 1.0 if self.age < 25 else max(0.5, 1 - (self.age - 25) * 0.01)
        self.vitesse = np.random.normal(50, 8) * facteur_age_vitesse
        if self.est_malade:
            self.force *= (1 - self.severite_maladie * 0.5)
            self.vitesse *= (1 - self.severite_maladie * 0.5)
        self.intelligence = np.random.normal(100, 15) + max(0, (self.age - 20) * 0.2)

        # Sauvegarde des stats max pour la récupération
        self.force_max = self.force
        self.vitesse_max = self.vitesse

    def _calculer_entrainement(self):
        self.niveau_entrainement = 0.0
        if 16 < self.age < 50:
            base_proba = 0.35 if self.sexe == 'Homme' else 0.20
            if random.random() < base_proba:
                self.niveau_entrainement = np.random.beta(2, 5)
                self.force += self.niveau_entrainement * 20
                self.vitesse += self.niveau_entrainement * 10
                self.force_max = self.force
                self.vitesse_max = self.vitesse

    def _calculer_richesse(self, region_data):
        base_region = region_data['eco_factor']
        if self.age < 18: facteur_age = 0.05
        elif self.age < 25: facteur_age = 0.2 + (self.age - 18) * 0.05
        elif self.age < 60: facteur_age = 0.6 + min(0.4, (self.age - 25) * 0.01)
        else: facteur_age = max(0.4, 1.0 - (self.age - 60) * 0.02)
        wealth = base_region * facteur_age * np.random.lognormal(mean=0, sigma=0.8)
        self.richesse = round(wealth, 2)
        if self.richesse < base_region * 0.1: index = 0
        elif self.richesse < base_region * 0.3: index = 1
        elif self.richesse < base_region * 0.5: index = 2
        elif self.richesse < base_region * 0.8: index = 3
        elif self.richesse < base_region * 1.2: index = 4
        elif self.richesse < base_region * 2.0: index = 5
        elif self.richesse < base_region * 4.0: index = 6
        elif self.richesse < base_region * 10.0: index = 7
        elif self.richesse < base_region * 50.0: index = 8
        else: index = 9
        self.classe_sociale = self.CLASSES_SOCIALES[index]

    def _initialiser_corps(self):
        base_pv = 20 + (self.poids / 2)
        self.corps = {
            'tete':     {'pv_max': round(base_pv * 0.5), 'pv_actuels': round(base_pv * 0.5)},
            'torse':    {'pv_max': round(base_pv * 1.0), 'pv_actuels': round(base_pv * 1.0)},
            'bras_g':   {'pv_max': round(base_pv * 0.4), 'pv_actuels': round(base_pv * 0.4)},
            'bras_d':   {'pv_max': round(base_pv * 0.4), 'pv_actuels': round(base_pv * 0.4)},
            'jambe_g':  {'pv_max': round(base_pv * 0.5), 'pv_actuels': round(base_pv * 0.5)},
            'jambe_d':  {'pv_max': round(base_pv * 0.5), 'pv_actuels': round(base_pv * 0.5)}
        }
        self.pv_total = sum(m['pv_actuels'] for m in self.corps.values())

    def _verifier_capacite_combat(self):
        capable = True
        if self.age < 8: capable = False
        if self.age > 85 and random.random() < 0.8: capable = False
        if self.severite_maladie > 0.8: capable = False
        self.peut_se_battre = capable

    def __str__(self):
        return (f"ID:{self.id:03d} {self.prenom:10s} ({self.age}ans, {self.sexe[0]}) | For:{self.force:.0f} Vit:{self.vitesse:.0f} | {self.classe_sociale}")

## --- 2. FONCTIONS DE SIMULATION ---


In [35]:
def combat_a_mort(combattant_1, combattant_2, verbose=True):
    # Vérifications pré-combat
    if not combattant_1.peut_se_battre:
        combattant_1.corps['tete']['pv_actuels'] = -50
        return combattant_2
    if not combattant_2.peut_se_battre:
        combattant_2.corps['tete']['pv_actuels'] = -50
        return combattant_1

    if verbose:
        print(f"--- DUEL : {combattant_1.prenom} (M:{combattant_1.mental:.0%}) VS {combattant_2.prenom} (M:{combattant_2.mental:.0%}) ---")

    # Variables de simulation
    timer_c1, timer_c2 = 0.0, 0.0

    while True:
        # Détermination de l'attaquant
        if timer_c1 <= timer_c2:
            attaquant, defenseur = combattant_1, combattant_2
            timer_att = timer_c1
            is_c1 = True
        else:
            attaquant, defenseur = combattant_2, combattant_1
            timer_att = timer_c2
            is_c1 = False

        # Calcul du temps d'action
        base_time = 1.5 / (attaquant.vitesse / 50.0) if attaquant.vitesse > 0 else 3.0
        # Le stress (mental bas) peut paralyser ou rendre frénétique, ici on ralentit
        if attaquant.mental < 0.3: base_time *= 1.5

        if is_c1: timer_c1 += base_time
        else: timer_c2 += base_time
        temps_combat = timer_att

        # --- NOUVEAU : CRISE DE PANIQUE ---
        # Si le mental est brisé, le combattant ne réagit plus
        if attaquant.mental <= 0.1:
            if verbose: print(f"[{temps_combat:.1f}s] {attaquant.prenom} est tétanisé par la terreur !")
            continue # Il passe son tour

        # --- Choix de la cible ---
        cibles = ['tete', 'torse', 'bras_g', 'bras_d', 'jambe_g', 'jambe_d']
        poids = [0.1, 0.4, 0.1, 0.1, 0.15, 0.15]

        # Si le mental est bas, on vise moins bien (coups désespérés)
        if attaquant.mental < 0.5:
            poids = [0.2, 0.3, 0.1, 0.1, 0.15, 0.15] # Plus de risques de toucher la tête (coup de chance/danger)

        cible = random.choices(cibles, weights=poids, k=1)[0]

        # --- Calcul de la touche ---
        # Le mental influence la précision
        precision_base = 0.7 + (attaquant.intelligence / 300) - (defenseur.vitesse / 300)
        precision_base *= (0.5 + 0.5 * attaquant.mental) # Malus si mental bas

        if random.random() < precision_base:
            # COUP TOUCHÉ
            degats = attaquant.force * random.uniform(0.8, 1.2)
            if cible == 'tete': degats *= 2.5
            elif cible == 'torse': degats *= 1.0
            else: degats *= 0.6

            degats_finaux = round(degats)
            defenseur.corps[cible]['pv_actuels'] -= degats_finaux

            if verbose:
                print(f"[{temps_combat:.1f}s] {attaquant.prenom} frappe {defenseur.prenom} au {cible} ({degats_finaux} dmg).")

            # --- NOUVEAU : IMPACT PSYCHOLOGIQUE ---
            # 1. La douleur affecte le mental du défenseur
            defenseur.mental -= 0.02 * (degats_finaux / 10) # Plus ça fait mal, plus ça traumatisé
            defenseur.mental = max(0, defenseur.mental)

            # 2. Voir le sang / blesser affecte l'attaquant (sauf s'il est entraîné)
            if attaquant.niveau_entrainement < 0.3: # Les non-aguers sont choqués
                attaquant.mental -= 0.01
            else:
                attaquant.mental -= 0.002 # Les guerriers sont endurcis

            # Effets physiques
            if defenseur.corps[cible]['pv_actuels'] <= 0:
                if 'jambe' in cible: defenseur.vitesse *= 0.5
                elif 'bras' in cible: defenseur.force *= 0.7

                # Choc de l'amputation/blessure grave
                defenseur.mental -= 0.15
                if verbose: print(f"   >> {defenseur.prenom} est gravement blessé ! Panique...")

            # Vérification Mort Physique
            if defenseur.corps['tete']['pv_actuels'] <= 0 or defenseur.corps['torse']['pv_actuels'] <= -20:
                winner = attaquant
                # Le gagnant est traumatisé par le meurtre
                winner.mental -= 0.1
                if verbose: print(f"--- MORT DE {defenseur.prenom} ---")
                break

            # Vérification Mort Psychologique (Suicide/Capitulation)
            if defenseur.mental <= 0:
                if verbose: print(f"--- {defenseur.prenom} s'effondre mentalement et se laisse mourir ---")
                defenseur.corps['tete']['pv_actuels'] = 0 # Suicide ou abandon total
                winner = attaquant
                break

        else:
            # Coup raté
            if verbose and random.random() < 0.2:
                print(f"[{temps_combat:.1f}s] {attaquant.prenom} manque son coup.")

    # Post-combat
    if winner:
        winner.pv_total = sum(m['pv_actuels'] for m in winner.corps.values())
        # Le survivant reprend ses esprits (adrénaline retombe)
        winner.mental = min(winner.mental_max, winner.mental + 0.05) # Léger apaisement
    return winner

In [36]:
def recuperation(individu, temps_heures, equipe_medecin):
    """
    Gère la récupération réaliste d'un individu après un combat.
    Inclut la guérison physique et mentale.
    """

    # --- 1. Calcul des taux de guérison ---

    # Taux Physique (PV/heure)
    # Naturellement, on guérit très lentement (0.1 PV/heure).
    # Avec médecins, c'est 5x plus rapide (désinfection, sutures).
    taux_naturel = 0.1
    taux_medical = 0.5
    taux_physique_reel = taux_medical if equipe_medecin else taux_naturel

    # Taux Mental (Points/heure)
    # La santé mentale se remonte plus lentement que les blessures légères.
    # Le repos aide (0.01 = 1% par heure). Les médecins (psychothérapie/médicaments) aident plus.
    taux_mental_naturel = 0.01
    taux_mental_medical = 0.05
    taux_mental_reel = taux_mental_medical if equipe_medecin else taux_mental_naturel

    # La douleur physique ralentit la guérison mentale
    pv_max_total = sum(m['pv_max'] for m in individu.corps.values())
    pv_actuel_total = sum(m['pv_actuels'] for m in individu.corps.values())
    ratio_sante = (pv_actuel_total / pv_max_total) if pv_max_total > 0 else 1.0

    # --- 2. Application de la guérison sur le corps ---
    pv_recup_total = temps_heures * taux_physique_reel

    for membre, data in individu.corps.items():
        pv_actuels = data['pv_actuels']
        pv_max = data['pv_max']

        # Si le membre est blessé
        if pv_actuels < pv_max:

            # Cas : Fracture ou Traumatisme grave (PV <= 0)
            if pv_actuels <= 0:
                if equipe_medecin:
                    # Stabilisation, chirurgie, plâtre
                    # On guérit le membre, mais cela prend du temps
                    data['pv_actuels'] += pv_recup_total

                    # Si on a passé assez de temps, le médecin fixe le problème
                    # On plafonne la guérison à 1 si le temps est court (juste stabilisé)
                    if data['pv_actuels'] < 1 and temps_heures >= 24:
                         data['pv_actuels'] = 1 # Le membre est "réparé" mais fragile
                    elif data['pv_actuels'] < 1:
                         data['pv_actuels'] = 0 # Toujours cassé mais pas infecté

                else:
                    # Sans médecin : Guérison naturelle d'une fracture quasi-nulle
                    # Le corps tente de souder, mais c'est long et risqué
                    data['pv_actuels'] += pv_recup_total * 0.1

            # Cas : Blessure légère (PV > 0)
            else:
                data['pv_actuels'] = min(pv_max, pv_actuels + pv_recup_total)

    # --- 3. Application de la guérison mentale ---
    # Le mental remonte, mais moins vite si on a mal (ratio_sante)
    # On utilise individu.mental_max si disponible, sinon 1.0
    max_mental = getattr(individu, 'mental_max', 1.0)
    recup_mentale = temps_heures * taux_mental_reel * ratio_sante
    individu.mental = min(max_mental, individu.mental + recup_mentale)

    # --- 4. Mise à jour des Stats dérivées (Force/Vitesse) ---
    # On recalcule la vitesse et la force en fonction de l'état des membres
    # On récupère les valeurs de référence (max)
    force_base = getattr(individu, 'force_max', individu.force)
    vitesse_base = getattr(individu, 'vitesse_max', individu.vitesse)

    # État moyen des paires de membres
    etat_jambes = (individu.corps['jambe_g']['pv_actuels'] + individu.corps['jambe_d']['pv_actuels']) / 2
    etat_bras = (individu.corps['bras_g']['pv_actuels'] + individu.corps['bras_d']['pv_actuels']) / 2

    pv_max_jambe = individu.corps['jambe_g']['pv_max']
    pv_max_bras = individu.corps['bras_g']['pv_max']

    # Logique de récupération fonctionnelle
    # Si le membre est toujours "cassé" (PV actuels <= 0), la stat reste pénalisée
    if etat_jambes <= 0:
        individu.vitesse = vitesse_base * 0.1 # Quasi immobile
    else:
        # La vitesse remonte proportionnellement à la guérison des jambes
        ratio_jambes = min(1.0, etat_jambes / pv_max_jambe)
        individu.vitesse = vitesse_base * ratio_jambes

    if etat_bras <= 0:
        individu.force = force_base * 0.2 # Très affaibli
    else:
        ratio_bras = min(1.0, etat_bras / pv_max_bras)
        individu.force = force_base * ratio_bras

    # --- 5. Mise à jour des totaux et capacité ---
    individu.pv_total = sum(m['pv_actuels'] for m in individu.corps.values())
    individu._verifier_capacite_combat() # Vérifie s'il peut toujours se battre

## --- 3. FONCTION HUNGER GAMES (Logique du tournoi) ---

In [54]:
def hunger_games(population, tracked_ids=None):
    if tracked_ids is None:
        tracked_ids = []

    round_num = 1
    historique_joueurs = {pid: "En vie" for pid in tracked_ids}

    while len(population) > 1:
        print(f"\n=== ROUND {round_num} | Survivants : {len(population)} ===")
        survivors = []
        random.shuffle(population)

        for i in range(0, len(population), 2):
            if i + 1 >= len(population):
                survivors.append(population[i])
                break

            h1 = population[i]
            h2 = population[i+1]

            # COMBAT A MORT
            winner = combat_a_mort(h1, h2, verbose=False)

            # RECUPERATION : 24h (1 jour)
            recuperation(winner, 24, False)

            survivors.append(winner)

            # --- VERIFICATION DES JOUEURS ET AFFICHAGE COMPARATIF ---
            if winner == h1:
                # h2 est le perdant
                if h2.id in tracked_ids:
                    print(f"\n>>> LE JOUEUR (ID:{h2.id}) A ÉTÉ ÉLIMINÉ AU ROUND {round_num} <<<")
                    print(f"   PERDANT : {h2}") # Affiche les stats du joueur
                    print(f"   GAGNANT : {h1}") # Affiche les stats du tueur pour comparaison
                    historique_joueurs[h2.id] = f"Mort Round {round_num} par {h1.prenom}"
            else:
                # h1 est le perdant
                if h1.id in tracked_ids:
                    print(f"\n>>> LE JOUEUR (ID:{h1.id}) A ÉTÉ ÉLIMINÉ AU ROUND {round_num} <<<")
                    print(f"   PERDANT : {h1}") # Affiche les stats du joueur
                    print(f"   GAGNANT : {h2}") # Affiche les stats du tueur pour comparaison
                    historique_joueurs[h1.id] = f"Mort Round {round_num} par {h2.prenom}"

        population = survivors

        # --- Affichage Graphiques ---
        if round_num == 1 or round_num == 10 or round_num == 20 or round_num >= 30:
            print(f"--- ANALYSE GRAPHIQUE DU ROUND {round_num} ---")
            # Décommentez les graphiques souhaités
            # graph_parite(population, round_num)
            # graph_force_vitesse(population, round_num)
            # graph_blessures(population, round_num)

        round_num += 1

    # --- BILAN FINAL ---
    champion = population[0]

    print(f"\n{'='*40}")
    print(f"LE VAINQUEUR EST : {champion}")
    print(f"{'='*40}")

    print(f"\n--- BILAN DES JOUEURS ---")
    for pid, status in historique_joueurs.items():
        if champion.id == pid:
            print(f"Joueur {pid} : VAINQUEUR DU TOURNOI !")
        else:
            print(f"Joueur {pid} : {status}")

    return champion

## 4 Génération des graphique

In [38]:
def graph_parite(population, round_num):
    """
    Affiche un camembert de la répartition H/F.
    """
    hommes = sum(1 for h in population if h.sexe == 'Homme')
    femmes = len(population) - hommes

    labels = ['Hommes', 'Femmes']
    sizes = [hommes, femmes]
    colors = ['lightskyblue', 'lightcoral']

    plt.figure(figsize=(6, 6))
    plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    plt.title(f"% d'hommes et de femmes au round {round_num}")
    plt.show()

In [39]:
def graph_entrainement(population, round_num):
    """
    Affiche la distribution du niveau d'entraînement (skill) des survivants.
    """
    niveaux = [h.niveau_entrainement for h in population]

    plt.figure(figsize=(8, 5))
    plt.hist(niveaux, bins=20, color='green', alpha=0.7, edgecolor='black')
    plt.xlabel("Niveau d'entraînement (0 = Débutant, 1 = Expert)")
    plt.ylabel("Nombre d'individus")
    plt.title(f"Compétence de combat au round {round_num}")
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.show()

In [40]:
def graph_age(population, round_num):
    """
    Affiche un histogramme des âges.
    """
    ages = [h.age for h in population]

    plt.figure(figsize=(8, 5))
    plt.hist(ages, bins=range(0, 110, 5), color='purple', alpha=0.7, edgecolor='black', rwidth=0.85)
    plt.xlabel("Âge")
    plt.ylabel("Effectif")
    plt.title(f"Pyramide des âges au round {round_num}")
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.show()

In [41]:
def graph_blessures(population, round_num):
    """
    Affiche l'état de santé moyen basé sur le % de PV restants.
    """
    etats = []
    for h in population:
        pv_max_total = sum(m['pv_max'] for m in h.corps.values())
        pv_actuel_total = sum(m['pv_actuels'] for m in h.corps.values())
        pourcentage = (pv_actuel_total / pv_max_total) * 100 if pv_max_total > 0 else 0
        etats.append(pourcentage)

    plt.figure(figsize=(8, 5))
    plt.hist(etats, bins=20, color='red', alpha=0.7, edgecolor='black')
    plt.xlabel("Pourcentage de vie restante (%)")
    plt.ylabel("Nombre de survivants")
    plt.title(f"État de santé de la population au round {round_num}")
    plt.axvline(x=50, color='black', linestyle='--', label='Seuil critique (50%)')
    plt.legend()
    plt.show()

In [42]:
def graph_force_vitesse(population, round_num):
    forces = [h.force for h in population]
    vitesses = [h.vitesse for h in population]

    plt.figure(figsize=(8, 6))
    plt.scatter(forces, vitesses, alpha=0.6, c='blue', edgecolors='w', linewidth=0.5)
    plt.xlabel("Force")
    plt.ylabel("Vitesse")
    plt.title(f"Profil de combat (Force vs Vitesse) au round {round_num}")
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

In [43]:
def graph_etat_mental(population, round_num):
    mentaux = [h.mental for h in population]

    plt.figure(figsize=(8, 5))
    plt.hist(mentaux, bins=20, color='indigo', alpha=0.7, edgecolor='black')
    plt.xlabel("Santé Mentale (0 = Brisé, 1 = Stable)")
    plt.ylabel("Nombre d'individus")
    plt.title(f"État psychologique au round {round_num}")
    plt.axvline(x=0.3, color='red', linestyle='--', label='Seuil de panique')
    plt.legend()
    plt.show()

In [44]:
def graph_regions(population, round_num):
    regions = [h.region_key for h in population]
    unique_regions = list(set(regions))
    counts = [regions.count(r) for r in unique_regions]

    plt.figure(figsize=(10, 5))
    plt.bar(unique_regions, counts, color='teal', edgecolor='black')
    plt.xlabel("Région d'origine")
    plt.ylabel("Nombre de survivants")
    plt.title(f"Survivants par région au round {round_num}")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [45]:
def graph_richesse(population, round_num):
    richesses = [h.richesse for h in population]

    plt.figure(figsize=(8, 5))
    plt.hist(richesses, bins=50, color='gold', alpha=0.7, edgecolor='black')
    plt.xlabel("Richesse ($)")
    plt.ylabel("Nombre de survivants")
    plt.title(f"Distribution des richesses au round {round_num}")
    plt.yscale('log') # Souvent utile car les riches sont très riches
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.show()

## 5 Tournoi avec moi

In [46]:
def integrer_joueur(population, prenom, age, sexe, taille, poids, mental, niveau_entrainement, intelligence=100):
    """
    Crée un joueur personnalisé en définissant son physique (Taille/Poids).
    La Force et la Vitesse sont calculées automatiquement.
    """
    # 1. Création de la structure de base
    joueur = Human()

    # 2. Identité et Physique imposés
    joueur.prenom = prenom
    joueur.age = age
    joueur.sexe = sexe
    joueur.taille = taille
    joueur.poids = poids

    # 3. Calcul automatique des Stats de combat (basé sur le modèle Human)
    # Force dépend du poids
    joueur.force = np.random.normal(30, 10) + (joueur.poids * 0.4)

    # Vitesse dépend de l'âge
    if joueur.age < 25:
        facteur_age_vitesse = 1.0
    else:
        facteur_age_vitesse = max(0.5, 1 - (joueur.age - 25) * 0.01)
    joueur.vitesse = np.random.normal(50, 8) * facteur_age_vitesse

    # Sauvegarde des maxs
    joueur.force_max = joueur.force
    joueur.vitesse_max = joueur.vitesse

    # 4. Mental et Compétences
    joueur.mental = mental
    joueur.mental_max = mental
    joueur.niveau_entrainement = niveau_entrainement
    joueur.intelligence = intelligence

    # Application du bonus d'entraînement (comme dans la classe Human)
    joueur.force += joueur.niveau_entrainement * 20
    joueur.vitesse += joueur.niveau_entrainement * 10
    joueur.force_max = joueur.force
    joueur.vitesse_max = joueur.vitesse

    # 5. Recalcul des PV du corps en fonction du NOUVEAU poids
    # (Sinon on aurait les PV d'un humain random)
    base_pv = 20 + (joueur.poids / 2)
    joueur.corps = {
        'tete':     {'pv_max': round(base_pv * 0.5), 'pv_actuels': round(base_pv * 0.5)},
        'torse':    {'pv_max': round(base_pv * 1.0), 'pv_actuels': round(base_pv * 1.0)},
        'bras_g':   {'pv_max': round(base_pv * 0.4), 'pv_actuels': round(base_pv * 0.4)},
        'bras_d':   {'pv_max': round(base_pv * 0.4), 'pv_actuels': round(base_pv * 0.4)},
        'jambe_g':  {'pv_max': round(base_pv * 0.5), 'pv_actuels': round(base_pv * 0.5)},
        'jambe_d':  {'pv_max': round(base_pv * 0.5), 'pv_actuels': round(base_pv * 0.5)}
    }
    joueur.pv_total = sum(m['pv_actuels'] for m in joueur.corps.values())

    # Vérification finale
    joueur._verifier_capacite_combat()

    # 6. Insertion aléatoire
    index_aleatoire = random.randint(0, len(population) - 1)
    population[index_aleatoire] = joueur

    print(f">>> JOUEUR AJOUTÉ : {prenom} (ID:{joueur.id}) | {taille}cm, {poids}kg | For calculée:{joueur.force:.0f} | Vit calculée:{joueur.vitesse:.0f}")

    return joueur.id

In [58]:
if __name__ == "__main__":
    print("--- Génération d'une population (65536 Humains) ---")
    population = []

    for _ in range(2048):
        h = Human()
        population.append(h)

    print(f"Population générée : {len(population)} individus.")

    # --- INTEGRATION DES 2 JOUEURS ---
    ids_joueurs = []

    # JOUEUR 1 : Un "Colosse" (Lourd, Grand -> Fort mais lent)
    ids_joueurs.append(integrer_joueur(
        population,
        prenom="Pierre",
        age=26,
        sexe="Homme",
        taille=190,       # Très grand
        poids=94,        # Très lourd -> Force énorme calculée
        mental=0.7,
        niveau_entrainement=0.5,
        intelligence=110
    ))

    # JOUEUR 2 : Un "Vif" (Léger, Petit -> Faible mais rapide)
    ids_joueurs.append(integrer_joueur(
        population,
        prenom="Alice",
        age=24,
        sexe="Femme",
        taille=157,       # Taille moyenne
        poids=49,         # Léger -> Force faible calculée, mais vitesse max
        mental=1.0,
        niveau_entrainement=0.7, # Très bon technique pour compenser la force
        intelligence=149
    ))

    print("\nLancement du tournoi Hunger Games...")

    # On passe la liste des IDs à surveiller
    vainqueur = hunger_games(population, tracked_ids=ids_joueurs)

    print(f"\nDétails du champion : {vainqueur}")

--- Génération d'une population (65536 Humains) ---
Population générée : 2048 individus.
>>> JOUEUR AJOUTÉ : Pierre (ID:80915) | 190cm, 94kg | For calculée:74 | Vit calculée:52
>>> JOUEUR AJOUTÉ : Alice (ID:80916) | 157cm, 49kg | For calculée:51 | Vit calculée:63

Lancement du tournoi Hunger Games...

=== ROUND 1 | Survivants : 2048 ===
--- ANALYSE GRAPHIQUE DU ROUND 1 ---

=== ROUND 2 | Survivants : 1024 ===

>>> LE JOUEUR (ID:80915) A ÉTÉ ÉLIMINÉ AU ROUND 2 <<<
   PERDANT : ID:80915 Pierre     (26ans, H) | For:74 Vit:34 | Indigent
   GAGNANT : ID:80496 Yui        (26ans, F) | For:39 Vit:13 | Modeste

=== ROUND 3 | Survivants : 512 ===

=== ROUND 4 | Survivants : 256 ===

=== ROUND 5 | Survivants : 128 ===

=== ROUND 6 | Survivants : 64 ===

>>> LE JOUEUR (ID:80916) A ÉTÉ ÉLIMINÉ AU ROUND 6 <<<
   PERDANT : ID:80916 Alice      (24ans, F) | For:51 Vit:61 | Pauvre
   GAGNANT : ID:80043 Kenji      (59ans, H) | For:70 Vit:27 | Modeste

=== ROUND 7 | Survivants : 32 ===

=== ROUND 8 | Surv